# Dataset Exploration and Analysis

This notebook explores the structure and characteristics of our plant disease datasets.

In [ ]:
# Import necessary libraries (run the setup from previous notebook first)
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from collections import Counter
import cv2
from PIL import Image

# Set paths
PROJECT_ROOT = Path('/Users/debabratapattnayak/web-dev/greencast')
DATASET_PATH = PROJECT_ROOT / 'dataset'
PLANTVILLAGE_PATH = DATASET_PATH / 'plantvillage dataset'
DATA2_PATH = DATASET_PATH / 'data2'

## Dataset Structure Analysis

In [ ]:
def analyze_dataset_structure(dataset_path):
    """Analyze the structure of the dataset"""
    structure = {}
    
    for root, dirs, files in os.walk(dataset_path):
        level = root.replace(str(dataset_path), '').count(os.sep)
        indent = ' ' * 2 * level
        folder_name = os.path.basename(root)
        
        if level <= 3:  # Limit depth for readability
            print(f"{indent}{folder_name}/ ({len(files)} files, {len(dirs)} subdirs)")
        
        # Count image files
        image_files = [f for f in files if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
        if image_files:
            structure[root] = len(image_files)
    
    return structure

print("=== PlantVillage Dataset Structure ===")
plantvillage_structure = analyze_dataset_structure(PLANTVILLAGE_PATH)

print("\n=== Data2 Dataset Structure ===")
data2_structure = analyze_dataset_structure(DATA2_PATH)

## Class Distribution Analysis

In [ ]:
def get_class_distribution(dataset_path, dataset_name):
    """Get class distribution for a dataset"""
    class_counts = {}
    
    for root, dirs, files in os.walk(dataset_path):
        # Count image files in each directory
        image_files = [f for f in files if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
        
        if image_files and len(image_files) > 10:  # Only consider directories with substantial images
            class_name = os.path.basename(root)
            class_counts[class_name] = len(image_files)
    
    # Create DataFrame
    df = pd.DataFrame(list(class_counts.items()), columns=['Class', 'Count'])
    df = df.sort_values('Count', ascending=False)
    
    print(f"\n=== {dataset_name} Class Distribution ===")
    print(f"Total classes: {len(df)}")
    print(f"Total images: {df['Count'].sum()}")
    print(f"Average images per class: {df['Count'].mean():.1f}")
    print(f"Min images per class: {df['Count'].min()}")
    print(f"Max images per class: {df['Count'].max()}")
    
    return df

# Analyze PlantVillage color dataset
plantvillage_color_path = PLANTVILLAGE_PATH / 'color'
if plantvillage_color_path.exists():
    pv_color_dist = get_class_distribution(plantvillage_color_path, "PlantVillage Color")
    
    # Plot distribution
    plt.figure(figsize=(15, 8))
    plt.subplot(1, 2, 1)
    top_20 = pv_color_dist.head(20)
    plt.barh(range(len(top_20)), top_20['Count'])
    plt.yticks(range(len(top_20)), top_20['Class'], fontsize=8)
    plt.xlabel('Number of Images')
    plt.title('Top 20 Classes - PlantVillage Color')
    plt.gca().invert_yaxis()
    
    # Distribution histogram
    plt.subplot(1, 2, 2)
    plt.hist(pv_color_dist['Count'], bins=20, alpha=0.7)
    plt.xlabel('Number of Images per Class')
    plt.ylabel('Number of Classes')
    plt.title('Distribution of Images per Class')
    
    plt.tight_layout()
    plt.show()

# Analyze Data2 dataset
data2_dist = get_class_distribution(DATA2_PATH, "Data2")

## Image Quality Analysis

In [ ]:
def analyze_image_properties(dataset_path, sample_size=100):
    """Analyze image properties like size, format, quality"""
    image_info = []
    
    # Collect sample images
    all_images = []
    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                all_images.append(os.path.join(root, file))
    
    # Sample random images
    if len(all_images) > sample_size:
        sample_images = np.random.choice(all_images, sample_size, replace=False)
    else:
        sample_images = all_images
    
    print(f"Analyzing {len(sample_images)} sample images...")
    
    for img_path in sample_images:
        try:
            # Load image
            img = cv2.imread(img_path)
            if img is not None:
                height, width, channels = img.shape
                file_size = os.path.getsize(img_path) / 1024  # KB
                
                # Calculate image statistics
                mean_brightness = np.mean(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY))
                
                image_info.append({
                    'width': width,
                    'height': height,
                    'channels': channels,
                    'file_size_kb': file_size,
                    'aspect_ratio': width / height,
                    'mean_brightness': mean_brightness,
                    'total_pixels': width * height
                })
        except Exception as e:
            print(f"Error processing {img_path}: {e}")
    
    # Create DataFrame
    df = pd.DataFrame(image_info)
    
    # Display statistics
    print("\n=== Image Properties Statistics ===")
    print(df.describe())
    
    return df

# Analyze PlantVillage color images
if plantvillage_color_path.exists():
    pv_img_props = analyze_image_properties(plantvillage_color_path)
    
    # Plot image properties
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # Width distribution
    axes[0, 0].hist(pv_img_props['width'], bins=20, alpha=0.7)
    axes[0, 0].set_title('Image Width Distribution')
    axes[0, 0].set_xlabel('Width (pixels)')
    
    # Height distribution
    axes[0, 1].hist(pv_img_props['height'], bins=20, alpha=0.7)
    axes[0, 1].set_title('Image Height Distribution')
    axes[0, 1].set_xlabel('Height (pixels)')
    
    # Aspect ratio
    axes[0, 2].hist(pv_img_props['aspect_ratio'], bins=20, alpha=0.7)
    axes[0, 2].set_title('Aspect Ratio Distribution')
    axes[0, 2].set_xlabel('Width/Height Ratio')
    
    # File size
    axes[1, 0].hist(pv_img_props['file_size_kb'], bins=20, alpha=0.7)
    axes[1, 0].set_title('File Size Distribution')
    axes[1, 0].set_xlabel('File Size (KB)')
    
    # Brightness
    axes[1, 1].hist(pv_img_props['mean_brightness'], bins=20, alpha=0.7)
    axes[1, 1].set_title('Mean Brightness Distribution')
    axes[1, 1].set_xlabel('Mean Brightness (0-255)')
    
    # Width vs Height scatter
    axes[1, 2].scatter(pv_img_props['width'], pv_img_props['height'], alpha=0.6)
    axes[1, 2].set_title('Width vs Height')
    axes[1, 2].set_xlabel('Width (pixels)')
    axes[1, 2].set_ylabel('Height (pixels)')
    
    plt.tight_layout()
    plt.show()

## Sample Images Visualization

In [ ]:
def visualize_sample_images(dataset_path, num_classes=6, images_per_class=4):
    """Visualize sample images from different classes"""
    
    # Get class directories
    class_dirs = []
    for item in os.listdir(dataset_path):
        item_path = os.path.join(dataset_path, item)
        if os.path.isdir(item_path):
            # Check if directory has images
            images = [f for f in os.listdir(item_path) 
                     if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
            if len(images) >= images_per_class:
                class_dirs.append((item, item_path, images))
    
    # Select random classes
    if len(class_dirs) > num_classes:
        selected_classes = np.random.choice(len(class_dirs), num_classes, replace=False)
        class_dirs = [class_dirs[i] for i in selected_classes]
    
    # Create visualization
    fig, axes = plt.subplots(num_classes, images_per_class, 
                            figsize=(images_per_class * 4, num_classes * 3))
    
    if num_classes == 1:
        axes = axes.reshape(1, -1)
    
    for class_idx, (class_name, class_path, images) in enumerate(class_dirs):
        # Select random images from this class
        selected_images = np.random.choice(images, images_per_class, replace=False)
        
        for img_idx, img_name in enumerate(selected_images):
            img_path = os.path.join(class_path, img_name)
            
            try:
                # Load and display image
                img = cv2.imread(img_path)
                img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                
                axes[class_idx, img_idx].imshow(img_rgb)
                axes[class_idx, img_idx].axis('off')
                
                if img_idx == 0:
                    axes[class_idx, img_idx].set_ylabel(class_name, 
                                                       rotation=90, 
                                                       fontsize=10)
                    
            except Exception as e:
                axes[class_idx, img_idx].text(0.5, 0.5, f'Error loading\n{img_name}', 
                                             ha='center', va='center')
                axes[class_idx, img_idx].axis('off')
    
    plt.suptitle('Sample Images from Different Classes', fontsize=16)
    plt.tight_layout()
    plt.show()

# Visualize PlantVillage color samples
if plantvillage_color_path.exists():
    print("Sample images from PlantVillage Color dataset:")
    visualize_sample_images(plantvillage_color_path)